In [1]:
import optuna

In [3]:
def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)#将变量'x'标记为超参，从-10到10的uniform分布里采值
    return (x - 2) ** 2#objective函数返回的是需要maximize or minimize的数值

In [ ]:
study = optuna.create_study()#创建一个study对象
study.optimize(objective, n_trials=100)#objective当作参数传入study的optimize函数实现对超参的自动搜索

[I 2021-05-12 11:07:33,703] A new study created in memory with name: no-name-356397c4-a659-49d8-8dcb-ee40dc210526
[I 2021-05-12 11:07:33,705] Trial 0 finished with value: 117.86084370995543 and parameters: {'x': -8.856373414264795}. Best is trial 0 with value: 117.86084370995543.
[I 2021-05-12 11:07:33,706] Trial 1 finished with value: 2.1632478798795964 and parameters: {'x': 0.5292016182088055}. Best is trial 1 with value: 2.1632478798795964.
[I 2021-05-12 11:07:33,707] Trial 2 finished with value: 0.7726499240713625 and parameters: {'x': 1.1209949237510841}. Best is trial 2 with value: 0.7726499240713625.
[I 2021-05-12 11:07:33,708] Trial 3 finished with value: 22.435134888123713 and parameters: {'x': 6.736574172133665}. Best is trial 2 with value: 0.7726499240713625.
[I 2021-05-12 11:07:33,708] Trial 4 finished with value: 56.056091833904404 and parameters: {'x': -5.487061628830392}. Best is trial 2 with value: 0.7726499240713625.
[I 2021-05-12 11:07:33,709] Trial 5 finished with va

In [5]:
study.best_params

{'x': 2.0024084804485813}

In [5]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import optuna

DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [6]:
def define_model(trial):#还没到optuna优化，就是定义一个架构
    # We optimize the number of layers, hidden untis and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)#优化隐藏层数量
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_uniform("dropout_l{}".format(i), 0.2, 0.5)#优化dropout比例
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)#*layers是一个list，必须加*转化


def get_mnist():
    # Load MNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.MNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

In [9]:
train_loader, valid_loader = get_mnist()

In [15]:
list(train_loader)[0][0].view(-1, 28 * 28)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [7]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the MNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:#取前30个的意思
                break

            data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)

            # Zeroing out gradient buffers.
            optimizer.zero_grad()
            # Performing a forward pass.
            output = model(data)
            # Computing negative Log Likelihood loss.
            loss = F.nll_loss(output, target)
            # Performing a backward pass.
            loss.backward()
            # Updating the weights.
            optimizer.step()

    # Validation of the model.
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            # Limiting validation data.
            if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                break
            data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability.
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / N_VALID_EXAMPLES
    return accuracy

In [8]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-05-12 11:34:00,922] A new study created in memory with name: no-name-87e9791f-1e1b-4bac-be83-63f8f8f933e4


Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting C:\Users\xuhan\Desktop\我的坚果云\Sys_learning\7_\MNIST\raw\train-images-idx3-ubyte.gz to C:\Users\xuhan\Desktop\我的坚果云\Sys_learning\7_\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/28881 [00:00<?, ?it/s]

...

Extracting C:\Users\xuhan\Desktop\我的坚果云\Sys_learning\7_\MNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\xuhan\Desktop\我的坚果云\Sys_learning\7_\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting C:\Users\xuhan\Desktop\我的坚果云\Sys_learning\7_\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\xuhan\Desktop\我的坚果云\Sys_learning\7_\MNIST\raw

Processing...
Done!


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
[I 2021-05-12 11:34:25,006] Trial 0 finished with value: 0.79609375 and parameters: {'n_layers': 3, 'n_units_l0': 118, 'dropout_l0': 0.26614664644012115, 'n_units_l1': 73, 'dropout_l1': 0.3687396403442701, 'n_units_l2': 31, 'dropout_l2': 0.4018811368663802, 'optimizer': 'Adam', 'lr': 0.0001903661732366321}. Best is trial 0 with value: 0.79609375.
[I 2021-05-12 11:34:28,418] Trial 1 finished with value

Number of finished trials:  100
Best trial:
  Value:  0.96015625
  Params: 
    n_layers: 1
    n_units_l0: 114
    dropout_l0: 0.32657701803048456
    optimizer: Adam
    lr: 0.016855132004642722
